In [1]:
from deepmol.loaders import CSVLoader
import pandas as pd

dataset = pd.read_csv("kegg_pathways_w_labels.csv", nrows=2)
labels = dataset.columns[4:]
# LOAD THE DATA
loader = CSVLoader('kegg_pathways_w_labels.csv', 
                   smiles_field='SMILES',labels_fields=labels)
dataset = loader.create_dataset(sep=",")

/home/jcapela/.local/share/mamba/envs/np_benchmark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-21 11:34:07,768 — INFO — Assuming multitask since y has more than one dimension. If otherwise, explicitly set the mode to 'classification' or 'regression'!


In [2]:
dataset.get_shape()

2025-07-21 11:34:07,776 — INFO — Mols_shape: (1255,)
2025-07-21 11:34:07,777 — INFO — Features_shape: None
2025-07-21 11:34:07,777 — INFO — Labels_shape: (1255, 22)


((1255,), None, (1255, 22))

In [3]:
from deepmol.splitters import MultiTaskStratifiedSplitter

new_splits = [[], [], [], [], []]

for i in range(5):
    seed = i
    datasets = MultiTaskStratifiedSplitter().k_fold_split(dataset, k=5, seed=seed)
    for train_dataset, test_dataset in datasets:
        train_dataset, validation_dataset = MultiTaskStratifiedSplitter().train_test_split(train_dataset, frac_train=0.8, seed=seed)
        new_splits[i].append((train_dataset, validation_dataset, test_dataset))



In [4]:
import pickle

# Specify the filename
filename = 'splits.pkl'

# Write the data to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(new_splits, file)

# Generate embeddings

In [5]:
import pickle


# open pickle file
# Specify the filename
filename = 'splits.pkl'
with open(filename, 'rb') as file:
    loaded_data = pickle.load(file)

In [6]:
import os
from deepmol.compound_featurization import LLM
from transformers import BertConfig, BertModel

from deepmol.standardizer import ChEMBLStandardizer

from deepmol.tokenizers import NPBERTTokenizer

transformer = LLM(model_path="NPBERT", model=BertModel, config_class=BertConfig,
                          tokenizer=NPBERTTokenizer(vocab_file=os.path.join("NPBERT", "vocab.txt")), device="cuda:0")


train_dataset, validation_dataset, test_dataset = loaded_data[0][0]
train_dataset = train_dataset.merge([validation_dataset, test_dataset])
ChEMBLStandardizer().standardize(train_dataset, inplace=True)
# Featurize the datasets
transformer.featurize(train_dataset, inplace=True)


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
2025-07-21 11:34:29.916034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-21 11:34:29.916088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-21 11:34:29.998089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-21 11:34:30.151405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild Tens

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
/home/jcapela/.local/share/mamba/envs/np_benchmark/lib/python3.10/site-packages/deepmol/compound_featurization/__init__.py:20: UserWarning: Mol2Vec not available. Please install it to use it. (pip install git+https://github.com/samoturk/mol2vec#egg=mol2vec)
  warnings.warn("Mol2Vec not available. Please install it to use it. "
Some weights of BertModel were not initialized from the model checkpoint at NPBERT/model.pt and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-07-21 11:34:55,589 — ERROR — Features are not the same length/type... Recalculate features for all inputs!
2025-07-21 11:34:55,591 — ERROR — Features are not the same length/type... Recalculate features for all inputs!
2025-07-21 11:34:55,592 — INFO — Standardizer ChEMBLStandardizer initialized with -1 jobs.


100%|██████████| 1255/1255 [00:10<00:00, 117.45it/s]


In [7]:
# Create the dictionary
data_dict = {id_: x for id_, x in zip(train_dataset.ids, train_dataset.X)}

In [8]:
# to pickle
import pickle

# Specify the filename
filename = 'np_bert.pkl'

# Write the data to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(data_dict, file)

In [9]:
# Load the data from the pickle file
with open(filename, 'rb') as file:
    np_bert = pickle.load(file)


In [10]:
import numpy as np

def featurize_dataset(dataset, np_bert):
    features = []
    for id_ in dataset.ids:
        features.append(np_bert[id_])
    features = np.array(features)
    dataset._X = features
    return dataset

In [11]:
new_splits = [[], [], [], [], []]

for i in range(5):
    for train_dataset, validation_dataset, test_dataset in loaded_data[i]:
        train_dataset = featurize_dataset(train_dataset, np_bert)
        validation_dataset = featurize_dataset(validation_dataset, np_bert)
        test_dataset = featurize_dataset(test_dataset, np_bert)
        new_splits[i].append((train_dataset, validation_dataset, test_dataset))

# Specify the filename
filename = 'splits_npbert.pkl'
# Write the data to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(new_splits, file)

In [12]:
test_dataset.X

array([[-0.00964477, -0.07120536, -0.3341364 , ...,  0.26339033,
        -0.42905506, -0.21124896],
       [ 0.0571223 , -0.06259119, -0.5391472 , ...,  0.12575962,
        -0.4212825 , -0.15620579],
       [ 0.13864245, -0.08747164, -0.61114496, ...,  0.0708169 ,
        -0.43497893, -0.15839529],
       ...,
       [ 0.08087426, -0.14390326, -0.5035606 , ...,  0.07976154,
        -0.40614304, -0.18456466],
       [ 0.17691417, -0.04101022, -0.37768027, ...,  0.21714582,
        -0.44233507, -0.18424739],
       [-0.09625542, -0.07367446, -0.26573285, ...,  0.2375433 ,
        -0.42717344, -0.18969171]])

In [13]:
import pickle


# open pickle file
# Specify the filename
filename = 'splits.pkl'
with open(filename, 'rb') as file:
    loaded_data = pickle.load(file)
    

In [16]:
import os
from deepmol.compound_featurization import LLM
from transformers import ModernBertModel, ModernBertConfig

from deepmol.standardizer import ChEMBLStandardizer


transformer = LLM(model_path="../ModernBERT", model=ModernBertModel, config_class=ModernBertConfig, device="cuda:2")


train_dataset, validation_dataset, test_dataset = loaded_data[0][0]
train_dataset = train_dataset.merge([validation_dataset, test_dataset])
ChEMBLStandardizer().standardize(train_dataset, inplace=True)
# Featurize the datasets
transformer.featurize(train_dataset, inplace=True)


2025-07-21 11:36:28,424 — ERROR — Features are not the same length/type... Recalculate features for all inputs!
2025-07-21 11:36:28,426 — ERROR — Features are not the same length/type... Recalculate features for all inputs!
2025-07-21 11:36:28,428 — INFO — Standardizer ChEMBLStandardizer initialized with -1 jobs.


100%|██████████| 1255/1255 [00:54<00:00, 23.12it/s]


In [17]:
train_dataset.get_shape()

2025-07-21 11:37:46,529 — INFO — Mols_shape: (1255,)
2025-07-21 11:37:46,530 — INFO — Features_shape: (1255, 1024)
2025-07-21 11:37:46,531 — INFO — Labels_shape: (1255, 22)


((1255,), (1255, 1024), (1255, 22))

In [20]:
# Create the dictionary
data_dict = {id_: x for id_, x in zip(train_dataset.ids, train_dataset.X)}

# to pickle
import pickle

# Specify the filename
filename = 'modern_bert.pkl'

# Write the data to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(data_dict, file)

In [21]:
# Load the data from the pickle file
with open(filename, 'rb') as file:
    np_bert = pickle.load(file)


In [22]:
new_splits = [[], [], [], [], []]

for i in range(5):
    for train_dataset, validation_dataset, test_dataset in loaded_data[i]:
        train_dataset = featurize_dataset(train_dataset, np_bert)
        validation_dataset = featurize_dataset(validation_dataset, np_bert)
        test_dataset = featurize_dataset(test_dataset, np_bert)
        new_splits[i].append((train_dataset, validation_dataset, test_dataset))

# Specify the filename
filename = 'splits_modern_bert.pkl'
# Write the data to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(new_splits, file)